In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import dask.dataframe as dd
from dask.distributed import Client
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42

if __name__ == '__main__':
    data_dir = '/Users/angam/mimic_project'  # Adjust if needed

    # Initialize Dask client
    client = Client(n_workers=6)
    print("Dask client initialized with 6 workers.")

    # Name of the file to save processed data
    processed_data_path = 'processed_data.pkl'

    # ---------------------------
    # Step 1: Load Core Tables and Define Target (Sepsis)
    # ---------------------------
    print("Loading core tables...")
    patients = pd.read_csv(os.path.join(data_dir, 'patients.csv'), usecols=['subject_id', 'gender', 'anchor_age', 'anchor_year_group'])
    admissions = pd.read_csv(os.path.join(data_dir, 'admissions.csv'), usecols=['subject_id', 'hadm_id', 'admittime', 'dischtime', 'hospital_expire_flag', 'insurance', 'marital_status'])
    icustays = pd.read_csv(os.path.join(data_dir, 'icustays.csv'), usecols=['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime'])

    data = admissions.merge(patients, on='subject_id', how='inner')
    data = data.merge(icustays, on=['subject_id', 'hadm_id'], how='inner')
    print("Merged data shape:", data.shape)

    # Load diagnoses for sepsis labeling
    diagnoses_icd = pd.read_csv(os.path.join(data_dir, 'diagnoses_icd.csv'), usecols=['subject_id', 'hadm_id', 'icd_code', 'icd_version'])
    d_icd_diagnoses = pd.read_csv(os.path.join(data_dir, 'd_icd_diagnoses.csv'), usecols=['icd_code', 'icd_version', 'long_title'])

    # Identify sepsis ICD codes (A41*)
    sepsis_codes = d_icd_diagnoses[
        (d_icd_diagnoses['icd_version'] == 10) &
        (d_icd_diagnoses['icd_code'].str.startswith('A41'))
    ]['icd_code'].unique()

    sepsis_stays = diagnoses_icd[
        (diagnoses_icd['icd_version'] == 10) &
        (diagnoses_icd['icd_code'].isin(sepsis_codes))
    ][['subject_id', 'hadm_id']].drop_duplicates()

    data['sepsis'] = data.apply(lambda row: 1 if ((row['subject_id'], row['hadm_id']) in sepsis_stays.set_index(['subject_id','hadm_id']).index) else 0, axis=1)
    print("Sepsis distribution:", data['sepsis'].value_counts(normalize=True))

    # ---------------------------
    # Step 2: Add Comorbidity Info (Number of distinct ICD diagnoses)
    # ---------------------------
    diagnosis_counts = diagnoses_icd.groupby(['subject_id', 'hadm_id'])['icd_code'].nunique().reset_index()
    diagnosis_counts.rename(columns={'icd_code': 'num_diagnoses'}, inplace=True)
    data = data.merge(diagnosis_counts, on=['subject_id', 'hadm_id'], how='left')
    data['num_diagnoses'] = data['num_diagnoses'].fillna(0)

    # ---------------------------
    # Step 3: Extract Vital Signs (First 24h)
    # ---------------------------
    icustays_subset = icustays[['subject_id', 'hadm_id', 'stay_id', 'intime']]
    icustays_subset['intime'] = pd.to_datetime(icustays_subset['intime'])

    vital_signs = {
        'Heart Rate': [211, 220045],
        'SysBP': [51, 220179],
        'DiasBP': [8368, 220180],
        'MeanBP': [52, 220181],
        'Respiratory Rate': [618, 220210],
        'Temperature': [676, 223761],
        'SpO2': [646, 220277]
    }

    print("Processing vital signs...")
    chartevents = dd.read_csv(os.path.join(data_dir, 'chartevents.csv'),
                              usecols=['subject_id', 'hadm_id', 'stay_id', 'itemid', 'charttime', 'valuenum'],
                              assume_missing=True, dtype={'valueuom': 'object'})

    vital_itemids = [item for sublist in vital_signs.values() for item in sublist]
    chartevents_filtered = chartevents[chartevents['itemid'].isin(vital_itemids)]
    chartevents_filtered['charttime'] = dd.to_datetime(chartevents_filtered['charttime'])
    chartevents_filtered = chartevents_filtered.merge(icustays_subset, on=['subject_id','hadm_id','stay_id'], how='inner')
    chartevents_filtered['hours_in'] = (chartevents_filtered['charttime'] - chartevents_filtered['intime']).dt.total_seconds()/3600
    chartevents_filtered = chartevents_filtered[(chartevents_filtered['hours_in'] >= 0) & (chartevents_filtered['hours_in'] <= 24)]

    d_items = pd.read_csv(os.path.join(data_dir, 'd_items.csv'), usecols=['itemid','label'])
    chartevents_filtered = chartevents_filtered.merge(d_items, on='itemid', how='left')

    vitals = chartevents_filtered.groupby(['subject_id','hadm_id','stay_id','label'])['valuenum'].mean().compute().reset_index()
    vitals_pivot = vitals.pivot_table(index=['subject_id','hadm_id','stay_id'], columns='label', values='valuenum').reset_index()

    # ---------------------------
    # Step 4: Extract Labs (WBC, Creatinine in first 24h)
    # ---------------------------
    print("Processing lab tests...")
    lab_tests = {
        'WBC': [51300],
        'Creatinine': [50912]
    }
    labevents = dd.read_csv(os.path.join(data_dir, 'labevents.csv'),
                            usecols=['subject_id', 'hadm_id', 'itemid', 'charttime', 'valuenum'],
                            assume_missing=True)
    lab_itemids = [item for sublist in lab_tests.values() for item in sublist]
    labevents_filtered = labevents[labevents['itemid'].isin(lab_itemids)]
    labevents_filtered['charttime'] = dd.to_datetime(labevents_filtered['charttime'])

    labevents_filtered = labevents_filtered.merge(icustays_subset, on=['subject_id','hadm_id'], how='inner')
    labevents_filtered['hours_in'] = (labevents_filtered['charttime'] - labevents_filtered['intime']).dt.total_seconds()/3600
    labevents_filtered = labevents_filtered[(labevents_filtered['hours_in'] >= 0) & (labevents_filtered['hours_in'] <= 24)]

    d_labitems = pd.read_csv(os.path.join(data_dir, 'd_labitems.csv'), usecols=['itemid','label'])
    labevents_filtered = labevents_filtered.merge(d_labitems, on='itemid', how='left')

    labs = labevents_filtered.groupby(['subject_id','hadm_id','stay_id','label'])['valuenum'].mean().compute().reset_index()
    labs_pivot = labs.pivot_table(index=['subject_id','hadm_id','stay_id'], columns='label', values='valuenum').reset_index()

    # Merge vitals and labs
    data = data.merge(vitals_pivot, on=['subject_id','hadm_id','stay_id'], how='left')
    data = data.merge(labs_pivot, on=['subject_id','hadm_id','stay_id'], how='left')

    # ---------------------------
    # Step 5: Preprocessing
    # ---------------------------
    categorical_features = ['gender', 'anchor_year_group', 'insurance', 'marital_status']
    data[categorical_features] = data[categorical_features].fillna('Unknown')
    data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

    data = data.rename(columns={'anchor_age': 'age'})

    exclude_cols = ['subject_id','hadm_id','stay_id','admittime','dischtime','intime','outtime','hospital_expire_flag','sepsis']
    numeric_cols = [col for col in data.columns if col not in exclude_cols and data[col].dtype in [np.float64, np.int64]]

    imputer = SimpleImputer(strategy='median')
    data[numeric_cols] = imputer.fit_transform(data[numeric_cols])

    scaler = StandardScaler()
    data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

    target = 'sepsis'
    data = data.dropna(subset=[target])
    X = data.drop(columns=exclude_cols)
    y = data[target].astype(int)

    joblib.dump((X, y), processed_data_path)
    print(f"Processed data saved to '{processed_data_path}'.")

    X_np = np.ascontiguousarray(X.values)
    y_np = np.ascontiguousarray(y.values)

    # ---------------------------
    # Step 6: Modeling
    # ---------------------------
    X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.3, random_state=RANDOM_STATE, stratify=y_np)

    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
        'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'Naive Bayes': GaussianNB(),
        'K-Nearest Neighbors': KNeighborsClassifier()
    }

    results = {}
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:,1] if hasattr(model, "predict_proba") else y_pred
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)
        f1 = f1_score(y_test, y_pred)
        results[model_name] = {'Accuracy': acc, 'AUC': auc, 'F1': f1}
        print(f"\n{model_name}:")
        print(f"Accuracy: {acc:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}")

    # Add SVM with reduced sample
    svm_model = SVC(probability=True, random_state=RANDOM_STATE, class_weight='balanced')
    svm_fraction = 0.3
    sample_size = int(len(X_train)*svm_fraction)
    np.random.seed(RANDOM_STATE)
    indices = np.random.choice(len(X_train), size=sample_size, replace=False)
    X_train_svm = X_train[indices]
    y_train_svm = y_train[indices]

    svm_model.fit(X_train_svm, y_train_svm)
    y_pred = svm_model.predict(X_test)
    y_pred_proba = svm_model.predict_proba(X_test)[:,1]
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    results['Support Vector Machine'] = {'Accuracy': acc, 'AUC': auc, 'F1': f1}

    print("\nSupport Vector Machine (SVM) with reduced training data:")
    print(f"Accuracy: {acc:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}")

    print("\nAll models evaluated successfully.")


In [ ]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from imblearn.under_sampling import RandomUnderSampler

import warnings

warnings.filterwarnings('ignore')

RANDOM_STATE = 42

########################################################################
# 1. Load Data
########################################################################
if __name__ == '__main__':
    print("Loading previously saved processed data...")
    X, y = joblib.load('processed_data.pkl')
    print("Data loaded. Shape of X:", X.shape, "Shape of y:", y.shape)

    if not isinstance(X, np.ndarray):
        X = X.values
    if not isinstance(y, np.ndarray):
        y = y.values

    ########################################################################
    # 2. Define Models and Noise Levels
    ########################################################################
    models = {
        'LR': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
        'DT': DecisionTreeClassifier(random_state=RANDOM_STATE),
        'RF': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'NB': GaussianNB(),
        'KNN': KNeighborsClassifier(),
        'SVM': SVC(probability=True, random_state=RANDOM_STATE, class_weight='balanced')
    }

    # Noise levels in ascending order: 0%, 5%, 10%, 15%, 20%, 25%, 30%, 35%, 40%
    noise_levels = np.arange(0.0, 0.45, 0.05)

    acc_results = {m: [] for m in models.keys()}
    tpr_results = {m: [] for m in models.keys()}
    tnr_results = {m: [] for m in models.keys()}

    ########################################################################
    # 3. Training, Evaluation, Confusion Matrices
    ########################################################################
    print("\nEvaluating all models for each noise level over 10 runs...\n")

    for model_key, model in models.items():
        print(f"--- {model_key} ---")
        for noise_level in noise_levels:
            sum_tn, sum_fp, sum_fn, sum_tp = 0, 0, 0, 0

            for iteration in range(10):
                random_seed = RANDOM_STATE + iteration
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.5, stratify=y, random_state=random_seed
                )

                y_train_noisy = y_train.copy()
                num_noisy = int(noise_level * len(y_train_noisy))
                np.random.seed(random_seed)
                noisy_indices = np.random.choice(len(y_train_noisy), size=num_noisy, replace=False)
                # Flip labels
                y_train_noisy[noisy_indices] = 1 - y_train_noisy[noisy_indices]

                rus = RandomUnderSampler(random_state=random_seed)
                X_train_bal, y_train_bal = rus.fit_resample(X_train, y_train_noisy)

                model.fit(X_train_bal, y_train_bal)

                y_pred = model.predict(X_test)

                tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=[0, 1]).ravel()
                sum_tn += tn
                sum_fp += fp
                sum_fn += fn
                sum_tp += tp


            total_runs = 10.0
            # Summed over 10 runs
            TN = sum_tn
            FP = sum_fp
            FN = sum_fn
            TP = sum_tp
            all_ = (TN + FP + FN + TP)

            TPR = TP / (TP + FN) if (TP + FN) > 0 else 0.0
            TNR = TN / (TN + FP) if (TN + FP) > 0 else 0.0
            ACC = (TP + TN) / all_ if all_ > 0 else 0.0

            acc_results[model_key].append(ACC * 100.0)
            tpr_results[model_key].append(TPR * 100.0)
            tnr_results[model_key].append(TNR * 100.0)

            print(f"Noise {int(noise_level * 100)}% -> Summed Confusion Matrix over 10 runs:")
            print(f"   [[TN={TN}, FP={FP}],")
            print(f"    [FN={FN}, TP={TP}]]")
            print(f"   Accuracy={ACC:.3f}, TPR={TPR:.3f}, TNR={TNR:.3f}")
        print()

    x_labels = np.array([100, 95, 90, 85, 80, 75, 70, 65, 60])
    n_groups = len(x_labels)


    for m in models.keys():
        acc_results[m].reverse()
        tpr_results[m].reverse()
        tnr_results[m].reverse()

    model_colors = {
        'LR': '#1f77b4',  # blue
        'DT': '#ff7f0e',  # orange
        'RF': '#2ca02c',  # green
        'NB': '#d62728',  # red
        'KNN': '#9467bd',  # purple
        'SVM': '#8c564b'  # brown
    }


    def plot_metric_sorted(dataset_name, metric_name, metric_data):
        """
        Plots a grouped bar chart for a given dataset and metric.
        Models are sorted in descending order based on their baseline (index 0) performance
        in the reversed arrays (which corresponds to 0% noise => 100% correct labels).

        X-axis: % accurate labels (descending order: 100, 95, …, 60)
        Y-axis: Metric performance (0–100%)
        """
        sorted_models = sorted(metric_data.keys(), key=lambda m: metric_data[m][0], reverse=True)
        n_models = len(sorted_models)

        x = np.arange(n_groups)
        bar_width = 0.12

        plt.figure(figsize=(12, 6))
        for i, model in enumerate(sorted_models):
            plt.bar(x + i * bar_width, metric_data[model],
                    width=bar_width,
                    color=model_colors.get(model, 'grey'),
                    label=model)

        plt.xlabel('% Accurate Labels (Descending)')
        plt.ylabel(f'{metric_name} (%)')
        plt.title(f'{dataset_name} - {metric_name} vs. % Accurate Labels\n(Models sorted by baseline performance)')
        plt.xticks(x + (n_models / 2 - 0.5) * bar_width, x_labels)
        plt.ylim(0, 100)
        plt.legend(title='Model')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()


    dataset_name = 'Sepsis Prediction (MIMIC-IV)'

    plot_metric_sorted(dataset_name, 'Accuracy', acc_results)

    plot_metric_sorted(dataset_name, 'TPR (Sensitivity)', tpr_results)


    plot_metric_sorted(dataset_name, 'TNR (Specificity)', tnr_results)

    print("All done!")